### C# Interactive Notebook - Carbon Example No.3

This example is a slightly more sophisticated version of C# example notebook no.2. A cross-tabulation report of brand popularity by month over a 5 year period is converted into a Microsoft DataFrame which is input to [Plotly.net](https://plotly.net/reference/index.html) to create a line chart.

In [1]:
#r "nuget: RCS.Carbon.Tables"
#r "nuget: RCS.Carbon.Licensing.RedCentre"
#r "nuget: DataView.InteractiveExtension,*-*"
#r "nuget: Microsoft.ML.DataView"
#r "nuget: Microsoft.Data.Analysis"
#r "nuget: Plotly.NET"
#r "nuget: Plotly.NET.Interactive"
#r "nuget: FSharp.Data"

using RCS.Carbon.Tables;
using RCS.Carbon.Variables;
using RCS.Carbon.Shared;
using RCS.Licensing.Provider.Shared;
using RCS.Licensing.Provider;

Installed Packages DataView.InteractiveExtension, 1.0.142 FSharp.Data, 6.6.0 Microsoft.Data.Analysis, 0.22.2 Microsoft.ML.DataView, 4.0.2 Plotly.NET, 5.1.0 Plotly.NET.Interactive, 5.0.0 RCS.Carbon.Licensing.RedCentre, 9.1.29 RCS.Carbon.Tables, 9.1.41

Loading extension script from `C:\Users\greg\.nuget\packages\dataview.interactiveextension\1.0.142\interactive-extensions\dotnet\extension.dib`

### Generate the cross-tabulation report

- The `LoginId` parameters are the Id and password of the Carbon guest user account which has sufficient data quota to process the large number of cases used in the example.
- Note that the report output has to be tweaked slightly to be in the exact format required by DataFrames. The X axis name is prefixed so that the X and Y axis labels border the rectangular data. The values '*' are converted to '0' because DataFrames expect numeric values.

In [2]:

var provider = new RedCentreLicensingProvider();
var engine = new CrossTabEngine(provider);
await engine.GetLicenceId("16499372", "C6H12O6");
engine.OpenJob("rcsruby", "demo");

var sprops  = new XSpecProperties(); 
sprops.InitAsMissing = true;

var dprops  = new XDisplayProperties();
dprops.Output.Format = XOutputFormat.TSV;
dprops.Cells.Frequencies.Visible = false;
dprops.Cells.RowPercents.Visible = true;    // Need this as Carbon table is transposed.
dprops.Cells.ColumnPercents.Visible = false;
dprops.Cells.PercentSign.Visible = false;
dprops.Decimals.Frequencies = 0;
dprops.Decimals.Expressions = 2;
dprops.Decimals.Statistics = 2;

dprops.Titles.Top.Visible = false;
dprops.Titles.Side.Visible = false;
dprops.Titles.Filter.Visible = false;
dprops.Titles.Weight.Visible = false;
dprops.Titles.Status.Visible = false;
dprops.Titles.Name.Visible = false;

dprops.Corner.Priority = XPriority.Top;

string chartXaxis ="month";
string seriesPlots = "uba";

string ret = engine.GenTab("Example 03", seriesPlots, chartXaxis + "(*)", null, null, sprops, dprops);

string strRet = ret.Replace('*','0');

Console.WriteLine(chartXaxis + strRet);

month	Brand1	Brand2	Brand3	Brand4	Brand5	Brand6	Brand7	Brand8	Brand9	Brand10
Jan2021	68.00	44.00	57.00	49.00	35.00	25.50	18.00	9.50	8.50	14.00
Feb2021	65.00	47.00	59.00	47.50	38.00	27.00	17.50	10.50	9.50	10.50
Mar2021	66.09	43.04	56.09	55.22	34.78	27.83	23.04	10.43	5.65	13.04
Apr2021	60.45	50.45	59.09	50.91	36.36	26.82	24.09	15.00	7.73	14.55
May2021	60.00	47.62	62.38	55.71	36.67	26.67	18.10	10.00	8.57	14.76
Jun2021	66.82	47.73	52.73	53.18	40.91	26.82	21.36	12.73	5.45	11.36
Jul2021	60.45	50.91	54.55	51.36	45.91	28.18	19.55	13.64	5.91	15.45
Aug2021	65.00	44.55	59.09	55.00	37.73	26.36	23.18	12.73	7.73	10.91
Sep2021	61.36	44.55	62.27	56.82	35.00	26.82	16.82	12.27	8.18	12.73
Oct2021	66.19	46.19	56.19	55.71	37.14	27.14	17.62	14.76	5.71	13.33
Nov2021	62.73	46.82	55.45	52.73	42.73	32.27	24.55	7.73	10.00	10.45
Dec2021	60.00	48.33	59.44	55.56	42.78	25.56	20.00	10.56	6.67	12.78
Jan2022	66.19	43.81	61.43	50.00	32.86	30.95	23.33	12.86	6.19	17.14
Feb2022	65.50	47.50	58.00	51.00	34.50	25.00	19.50	12.

### Convert Carbon data to a DataFrame

Also creates a list of chart series names

In [3]:
using Microsoft.Data.Analysis;
using Microsoft.ML;
using System.Collections.Generic;

var df1 = DataFrame.LoadCsvFromString(chartXaxis +  strRet, '\t');

var chartSeriesNames = df1.Columns.Skip(1).Select(c => c.Name).ToArray();   // Skip first 1 which is the label "month"
chartSeriesNames

[ Brand1, Brand2, Brand3, Brand4, Brand5, Brand6, Brand7, Brand8, Brand9, Brand10 ]

### Plotly.Net

API reference is https://plotly.net/reference/index.html

This produces an interactive inline chart. Click legend to toggle series on and off. Use Zoom and Pan to see detail.

In [4]:
#r "nuget: Plotly.NET, 5.1.0"
#r "nuget: Plotly.NET.Interactive, 5.0.0"
#r "nuget: Plotly.NET.CSharp, 0.13.0"
#r "nuget: Microsoft.Data.Analysis, 0.22.2"

using Plotly.NET;
using Plotly.NET.Interactive;
using Plotly.NET.LayoutObjects;
using Plotly.NET.CSharp;
using Microsoft.FSharp.Core;
using Microsoft.FSharp.Collections;
using Microsoft.Data.Analysis;
using System.Linq;
using System;
using System.Collections.Generic;

// Define axes
LinearAxis catAxis = new LinearAxis();
catAxis.SetValue("axistype", "Date");
catAxis.SetValue("title", chartXaxis);
catAxis.SetValue("zerolinecolor", "#ffff");
catAxis.SetValue("gridcolor", "#ffff");
catAxis.SetValue("showline", true);
catAxis.SetValue("zerolinewidth", 2);

// Set explicit x-axis range based on data
var xColumn = df1[chartXaxis];
var xData = new DateTime[xColumn.Length];
for (int i = 0; i < xColumn.Length; i++)
{
    xData[i] = xColumn[i] == null ? default(DateTime) : (DateTime)xColumn[i];
}
var validDates = xData.Where(d => d != default(DateTime)).ToArray();
var minDate = validDates.Any() ? validDates.Min() : DateTime.Now;
var maxDate = validDates.Any() ? validDates.Max() : DateTime.Now;
catAxis.SetValue("range", new[] { minDate, maxDate.AddDays(1) }); // Add 1 day for padding
catAxis.SetValue("autorange", false); // Disable autorange
catAxis.SetValue("tickformat", "%Y-%m-%d"); // Format date ticks

LinearAxis yAxis = new LinearAxis();
yAxis.SetValue("title", "Respondents");
yAxis.SetValue("zerolinecolor", "#ffff");
yAxis.SetValue("gridcolor", "#ffff");
yAxis.SetValue("showline", true);
yAxis.SetValue("zerolinewidth", 2);

// Define layout
Layout layout = new Layout();
layout.SetValue("xaxis", catAxis);
layout.SetValue("yaxis", yAxis);
layout.SetValue("title", "Data from Carbon");
layout.SetValue("plot_bgcolor", "#e5ecf6");
layout.SetValue("showlegend", true);
layout.SetValue("width", 1400); // ~15 inches at 80px/inch
layout.SetValue("height", 600); // Balanced height
layout.SetValue("margin", new { l = 120, r = 120, t = 100, b = 100 }); // ~1.5 inches left/right, 1.25 inches top/bottom

// Create traces
var traces = new List<Trace>();
for (int a = 0; a < chartSeriesNames.Length; a++)
{
    Trace t = new Trace("line");
    t.SetValue("x", df1[chartXaxis]);
    t.SetValue("y", df1[chartSeriesNames[a]]);
    t.SetValue("name", chartSeriesNames[a]);
    traces.Add(t);
}

// Create GenericChart using Chart.Line
var charts = traces.Select(t => {
    // Convert DateTime for x-axis
    var xDataLocal = new DateTime[xColumn.Length];
    for (int i = 0; i < xColumn.Length; i++)
    {
        xDataLocal[i] = xColumn[i] == null ? default(DateTime) : (DateTime)xColumn[i];
    }

    // Convert string to double for y-axis
    var yColumn = df1[chartSeriesNames[traces.IndexOf(t)]];
    var yData = new double[yColumn.Length];
    for (int i = 0; i < yColumn.Length; i++)
    {
        yData[i] = yColumn[i] == null ? 0.0 : double.Parse(yColumn[i].ToString(), System.Globalization.CultureInfo.InvariantCulture);
    }

    return Plotly.NET.CSharp.Chart.Line<DateTime, double, string>(
        x: xDataLocal,
        y: yData,
        Name: chartSeriesNames[traces.IndexOf(t)]
    );
});

// Combine charts and apply layout
var combinedChart = Plotly.NET.CSharp.Chart.Combine(charts)
    .WithLayout(layout);

// Display the chart
combinedChart

Installed Packages Microsoft.Data.Analysis, 0.22.2 Plotly.NET, 5.1.0 Plotly.NET.CSharp, 0.13.0 Plotly.NET.Interactive, 5.0.0

<!-- Plotly chart will be drawn inside this DIV -->

In [5]:
bool closed = engine.CloseJob();
Console.WriteLine($"Job closed = {closed} at {DateTime.Now:HH:mm:ss}");

Job closed = True at 12:38:49
